In [1]:
import sys
sys.path.append("/home/marin/projects/mhac/build/release")
import mhac

In [2]:
%matplotlib inline
import os, random, copy, math, csv, time
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mhac.set_log_level(mhac.LogLevel.off)

In [4]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [5]:
def nparray_to_timematrix(arr):
    tm = mhac.problems.jss.TimeMatrix()

    for time_list in arr.tolist():
        vint = mhac.VectorInt()
        for duration in time_list:
            vint.append(int(duration))
        tm.append(vint)

    return tm

In [6]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [7]:
def process_files(file_data, folder_path):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        # problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # model = mhac.physics.SimulatedAnnealing(problem)
        
        # problem = mhac.problems.jss.GA_JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # model = mhac.evolutionary.GeneticAlgorithm(problem)
        # model.setTournamentSize(15)

        problem = mhac.problems.jss.ACO_JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        model = mhac.swarm.AntColonyOptimization(problem)

        start_time = time.time()

        # sol = model.solve(100, 0.1, 0.995)
        # sol = model.solve(100, 30, 0.1, mhac.evolutionary.SelectionType.TOURNAMENT)

        sol = model.solve(100, 10, 0.7, 0.7, 0.25)

        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")
        print(f"Schedule: {sol.schedule} Cost: {sol.cost}")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [8]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [9]:
folder_path = "../../data/jss/imrg/ds1/testbed_1_s"
file_data = read_all_files_in_folder(folder_path)

In [10]:
results = process_files(file_data, folder_path)

Processing file: t1s_0238.txt took 0.0635s
Schedule: VectorInt[5, 9, 3, 2, 1, 7, 6, 8, 4, 0] Cost: 11776.0
Processing file: t1s_0216.txt took 0.0805s
Schedule: VectorInt[3, 6, 8, 7, 0, 9, 5, 2, 4, 1] Cost: 12387.0
Processing file: t1s_0312.txt took 0.0518s
Schedule: VectorInt[7, 5, 1, 9, 4, 8, 10, 0, 3, 6, 2, 11] Cost: 10986.0
Processing file: t1s_0281.txt took 0.0308s
Schedule: VectorInt[10, 1, 2, 8, 0, 4, 3, 6, 7, 5, 11, 9] Cost: 6917.0
Processing file: t1s_0113.txt took 0.0317s
Schedule: VectorInt[2, 0, 3, 4, 1] Cost: 4745.0
Processing file: t1s_0082.txt took 0.0200s
Schedule: VectorInt[3, 0, 1, 2, 4] Cost: 2629.0
Processing file: t1s_0192.txt took 0.0438s
Schedule: VectorInt[7, 1, 8, 4, 5, 6, 2, 0, 3, 9] Cost: 8296.0
Processing file: t1s_0266.txt took 0.0183s
Schedule: VectorInt[2, 0, 8, 1, 6, 4, 10, 5, 3, 11, 7, 9] Cost: 4666.0
Processing file: t1s_0021.txt took 0.0117s
Schedule: VectorInt[0, 1, 4, 2, 3] Cost: 886.0
Processing file: t1s_0018.txt took 0.0125s
Schedule: VectorInt[2,

In [89]:
results.sort(key=lambda x: x["filename"])
output_file = "results/mhac.csv"
write_results_to_csv(results, output_file)

# Results SA

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 318.20s |
| 1000 | 0.001 | 0.9995 | 239.10s |
| 100 | 0.1 | 0.9995 | 139.87s |
| 100 | 0.1 | 0.999 | 53.26s |
| 100 | 0.1 | 0.995 | 11.30s |

# Python problems and mhac

In [13]:
from queue import Queue

class PythonJSSP(mhac.common.Problem):
    def __init__(self, processing_times):
        super().__init__()
        self.processing_times = processing_times
        self.N = len(processing_times)
        self.M = len(processing_times[0])

    def generateInitialSolution(self):
        sol = mhac.problems.jss.JSSS()
        sol.schedule = random.sample(range(self.N), self.N)
        return sol

    def generateNewSolution(self, initialSol: mhac.problems.jss.JSSS):
        i, j = sorted(random.sample(range(self.N), 2))
        newSol = mhac.problems.jss.JSSS()
        newSol.schedule = initialSol.schedule  # copying with pybind, not using python refs
        # old_segment = newSol.tour[i:j+1]
        # newSol.tour[i:j+1] = old_segment[::-1]  # Slice and reverse the segment
        newSol.schedule[i], newSol.schedule[j] = newSol.schedule[j], newSol.schedule[i]
        return newSol

    def evaluateSolution(self, sol: mhac.problems.jss.JSSS):        
        # Initialize completion times matrix
        completion_times = [[0] * self.M for _ in range(self.N)]
        total_completion_time = 0

        for job_index in range(self.N):
            job = sol.schedule[job_index]

            for machine in range(self.M):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == self.M - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    

class PythonJSSP_GA(mhac.problems.jss.GA_JSSP, PythonJSSP):
    def __init__(self, processing_times):
        mhac.problems.jss.GA_JSSP.__init__(self, processing_times)
        PythonJSSP.__init__(self, processing_times)

    def repair(self, jss: mhac.problems.jss.JSSS):
        size = jss.getSize()

        seen = set()
        duplicates = []
        isPresent = [False] * size

        # Identify duplicates and check which elements are present
        for i in range(size):
            if jss.schedule[i] in seen:
                # This machine is a duplicate
                duplicates.append(i)
            else:
                seen.add(jss.schedule[i])
                isPresent[jss.schedule[i]] = True

        # Find missing elements
        missing = Queue()
        for i in range(size):
            if not isPresent[i]:
                missing.put(i)

        # Replace duplicates with missing elements
        for idx in duplicates:
            if not missing.empty():
                jss.schedule[idx] = missing.get()
    
    def crossover(self, parent1, parent2, outChild1, outChild2):
        # Clear existing schedules if necessary
        outChild1.schedule.clear()
        outChild2.schedule.clear()

        # Assuming schedule is a list within the JSSS objects
        cutPoint = random.randint(0, len(parent1.schedule) - 1)

        # Create children's schedules by slicing and appending parent schedules
        outChild1.schedule.extend(parent1.schedule[:cutPoint] + parent2.schedule[cutPoint:])
        outChild2.schedule.extend(parent2.schedule[:cutPoint] + parent1.schedule[cutPoint:])

        # Repair schedules
        self.repair(outChild1)
        self.repair(outChild2)
        
        # Evaluate costs
        outChild1.cost = self.evaluateSolution(outChild1)
        outChild2.cost = self.evaluateSolution(outChild2)

    def mutation(self, outChild, mutationChance):        
        i, j = random.sample(len(outChild.schedule), 2)

        if random.random() < mutationChance:
            outChild.schedule[i], outChild.schedule[j] = outChild.schedule[j], outChild.schedule[i]
            outChild.cost = self.evaluateSolution(outChild)


class PythonJSSP_ACO(mhac.problems.jss.ACO_JSSP, PythonJSSP):
    def __init__(self, processing_times):
        mhac.problems.jss.ACO_JSSP.__init__(self, processing_times)
        PythonJSSP.__init__(self, processing_times)

    def updateAntPath(self, ant, pm, alpha, beta):
        schedule_size = len(ant.schedule)
        probabilities = np.zeros(schedule_size)
        
        for i in range(schedule_size-1):
            current_job = ant.schedule[i]
            sum_probabilities = 0.0

            for j in range(i+1, schedule_size):
                next_job = ant.schedule[j]
                pheromone = pm[current_job][next_job]**alpha
                
                heuristic = 0 if (self.processing_times[current_job][next_job] == 0) else 1.0 / self.processing_times[current_job][next_job]

                eta = heuristic ** beta
                probabilities[j] = pheromone * eta

                sum_probabilities += probabilities[j]

            if sum_probabilities > 0:
                probabilities[i+1:schedule_size] /= sum_probabilities
                selected_index = random.choices(range(i + 1, schedule_size), weights=probabilities[i + 1:schedule_size])[0]
                ant.schedule[i + 1], ant.schedule[selected_index] = ant.schedule[selected_index], ant.schedule[i + 1]

    def updatePheromoneMatrix(self, ant, pm, rho):
        total_job_completion_time = self.evaluateSolution(ant)
        deposit = 1.0 / total_job_completion_time

        # Evaporate the existing pheromone
        size = len(pm)
        for i in range(size):
            for j in range(size):
                pm[i][j] *= (1 - rho)

        # Deposit new pheromones based on the ant's path
        for k in range(len(ant.schedule) - 1):
            i = ant.schedule[k]
            j = ant.schedule[k + 1]
            pm[i][j] += deposit


In [15]:
def process_files_pyjssp(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        # problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # SA = mhac.physics.SimulatedAnnealing(problem)

        # problem = PythonJSSP_GA(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # GA = mhac.evolutionary.GeneticAlgorithm(problem)
        # GA.setTournamentSize(5)

        problem = PythonJSSP_ACO(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        ACO = mhac.swarm.AntColonyOptimization(problem)

        start_time = time.time()

        # sol = SA.solve(100, 0.1, 0.995)
        # sol = GA.solve(200, 10, 0.1, mhac.evolutionary.SelectionType.TOURNAMENT)
        sol = ACO.solve(100, 10, 0.7, 0.7, 0.25)
        
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")
        print(f"Schedule: {sol.schedule} Cost: {sol.cost}")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [16]:
results = process_files_pyjssp(file_data)

Processing file: t1s_0238.txt took 0.5809s
Schedule: VectorInt[5, 2, 3, 9, 8, 6, 0, 4, 1, 7] Cost: 11052.0
Processing file: t1s_0216.txt took 0.6644s
Schedule: VectorInt[1, 6, 2, 4, 8, 7, 9, 3, 5, 0] Cost: 11998.0
Processing file: t1s_0312.txt took 0.7558s
Schedule: VectorInt[5, 4, 10, 11, 8, 9, 2, 3, 7, 6, 0, 1] Cost: 10356.0
Processing file: t1s_0281.txt took 0.4686s
Schedule: VectorInt[8, 6, 10, 1, 4, 0, 11, 2, 3, 5, 7, 9] Cost: 6487.0
Processing file: t1s_0113.txt took 0.2225s
Schedule: VectorInt[2, 0, 3, 4, 1] Cost: 4745.0
Processing file: t1s_0082.txt took 0.1662s
Schedule: VectorInt[3, 0, 1, 4, 2] Cost: 2624.0
Processing file: t1s_0192.txt took 0.4945s
Schedule: VectorInt[4, 1, 3, 8, 9, 5, 7, 6, 2, 0] Cost: 8032.0
Processing file: t1s_0266.txt took 0.3783s
Schedule: VectorInt[1, 5, 2, 3, 8, 0, 9, 10, 11, 7, 4, 6] Cost: 4158.0
Processing file: t1s_0021.txt took 0.1165s
Schedule: VectorInt[0, 1, 4, 2, 3] Cost: 886.0
Processing file: t1s_0018.txt took 0.1168s
Schedule: VectorInt[1,

# Results SA

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 2725.92s |
| 1000 | 0.001 | 0.9995 | 1751.22s |
| 100 | 0.1 | 0.9995 | 876.53s |
| 100 | 0.1 | 0.999 | 500.27s |
| 100 | 0.1 | 0.995 | 90.68s |
